In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import yaml
import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
import tensorflow as tf
from pathlib import Path
from time import strftime
from shutil import rmtree
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib
matplotlib.use("TKAgg", force=True)
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from rtapipe.lib.utils.misc import dotdict
from rtapipe.lib.dataset.data_manager import DataManager
from rtapipe.lib.datasource.Photometry3 import OnlinePhotometry, SimulationParams
from rtapipe.lib.plotting.plotting import plot_sequences
from rtapipe.lib.models.anomaly_detector_builder import AnomalyDetectorBuilder
from rtapipe.lib.evaluation.pval import get_pval_table, get_threshold_for_sigma
from rtapipe.lib.standardanalysis.li_ma import LiMa
from rtapipe.lib.evaluation.pval import get_pval_table, get_threshold_for_sigma, get_sigma_from_pvalue, get_sigma_for_ts_array, get_sigma_from_ts

# Parameters

In [3]:
INTEGRATION_TIME=5
SCALER="robust"
output_dir = "./logs/test_models_dataset_itime_5_c_robust_scaler_out"

# Loading the models

In [4]:
with open("./trained_models.yaml", "r") as f:
    try:
        configs = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)
configs = [dotdict(c) for c in configs["models"]]

In [7]:
for i, config in enumerate(configs):
    if      config.name     == "AnomalyDetector_rnn_l2_u32" \
        and config.patience == 5 \
        and config.integrationtime == INTEGRATION_TIME \
        and config.scaler == SCALER:
        print(config)
        break
model_config_cnn = configs[i]
model_config_cnn.ad = AnomalyDetectorBuilder.getAnomalyDetector(name=model_config_cnn.name, timesteps=model_config_cnn.timesteps, nfeatures=model_config_cnn.nfeatures, load_model="True", training_epoch_dir=model_config_cnn.path, training=False)
model_config_cnn.pvalue_table = get_pval_table(model_config_cnn.pval_path) 
model_config_cnn.name

{'patience': 5, 'integrationtime': 5, 'timesteps': 5, 'nfeatures': 3, 'scaler': 'robust', 'name': 'AnomalyDetector_rnn_l2_u32', 'path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221130-094250_it_5_pat_5_scaler_robust/model_AnomalyDetector_rnn_l2_u32_dataset_train_set_c_tsl_5_nbins_3_tsl_3600/epochs/epoch_11', 'epoch': 11, 'pval_path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221130-094250_it_5_pat_5_scaler_robust/model_AnomalyDetector_rnn_l2_u32_dataset_train_set_c_tsl_5_nbins_3_tsl_3600/epochs/epoch_11/pvalues/pval_20221130-130200/pvalue_bins_100_0.numpy.txt', 'scaler_path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221130-094250_it_5_pat_5_scaler_robust/model_AnomalyDetector_rnn_l2_u32_dataset_train_set_c_tsl_5_nbins_3_tsl_3600/fitted_scaler_robust_itime_5_tsl_3600.pickle'}
AnomalyDetector_rnn_l2_u32 - input shape: (5,3)


'AnomalyDetector_rnn_l2_u32'

In [8]:
for i, config in enumerate(configs):
    if      config.name     == "AnomalyDetector_rnn_l2_u32" \
        and config.patience == 5 \
        and config.integrationtime == INTEGRATION_TIME \
        and config.scaler == SCALER:
        
        break
model_config_rnn = configs[i]
model_config_rnn.ad = AnomalyDetectorBuilder.getAnomalyDetector(name=model_config_rnn.name, timesteps=model_config_rnn.timesteps, nfeatures=model_config_rnn.nfeatures, load_model="True", training_epoch_dir=model_config_rnn.path, training=False)
model_config_rnn.pvalue_table = get_pval_table(model_config_rnn.pval_path) 
model_config_rnn.name

AnomalyDetector_rnn_l2_u32 - input shape: (5,3)


'AnomalyDetector_rnn_l2_u32'

# Loading the test dataset

In [9]:
features_names = ["EB_0.04-0.117","EB_2-0.117-0.342","EB_0.342-1"]

In [10]:
data_manager = DataManager(output_dir)

In [11]:
dataset_id="test_itime_5_c_tsl_5_nbins_3"

In [12]:
dataset_folder = "/data01/homes/baroncelli/phd/rtapipe/scripts/ml/dataset_generation/test/itime_5_c/fits_data"
fits_files = DataManager.load_fits_data(dataset_folder, limit=210)
test_set_size = len(fits_files)

Loaded 209 files


In [13]:
sim_params = SimulationParams(runid=None, onset=250, emin=0.04, emax=1, tmin=0, tobs=500, offset=0.5, irf="North_z40_5h_LST", roi=2.5, caldb="prod5-v0.1", simtype="grb")
multiple_templates = True
add_target_region = True
integration_time = 5
number_of_energy_bins = 3
tsl = 100
threads = 30
normalize = True
#data_manager.transform_to_timeseries(fits_files, sim_params, add_target_region, integration_time=integration_time, number_of_energy_bins=number_of_energy_bins, tsl=tsl, normalize=normalize, threads=threads, multiple_templates=multiple_templates)
data_manager.load_saved_data(5, 100)

Loading cached data from run0002_ID000044_it_5_tsl_100.npy
Loading cached data from run0011_ID000139_it_5_tsl_100.npy
Loading cached data from run0016_ID000340_it_5_tsl_100.npy
Loading cached data from run0017_ID000132_it_5_tsl_100.npy
Loading cached data from run0017_ID000261_it_5_tsl_100.npy
Loading cached data from run0019_ID000070_it_5_tsl_100.npy
Loading cached data from run0019_ID000268_it_5_tsl_100.npy
Loading cached data from run0025_ID000471_it_5_tsl_100.npy
Loading cached data from run0026_ID000127_it_5_tsl_100.npy
Loading cached data from run0029_ID000397_it_5_tsl_100.npy
Loading cached data from run0029_ID000446_it_5_tsl_100.npy
Loading cached data from run0032_ID000190_it_5_tsl_100.npy
Loading cached data from run0038_ID000228_it_5_tsl_100.npy
Loading cached data from run0043_ID000231_it_5_tsl_100.npy
Loading cached data from run0043_ID000324_it_5_tsl_100.npy
Loading cached data from run0043_ID000460_it_5_tsl_100.npy
Loading cached data from run0046_ID000029_it_5_tsl_100.n

In [ ]:
assert len(data_manager.data) == test_set_size-1

## Plot some samples

In [ ]:
for template in list(data_manager.data.keys())[0:5]:
    data_manager.plot_timeseries(template, data_manager.data[template], 1, sim_params, output_dir, labels=features_names)

# TESTING

In [14]:
data_manager.load_scaler(model_config_rnn.scaler_path) # model_config_rnn.scaler_path or model_config_cnn.scaler_path

In [15]:
test_all_x, test_all_y = data_manager.get_test_set_all_templates(verbose=False, onset=250, integration_time=integration_time, sub_window_size=5, stride=1)

[2022-12-05 16:01:18.802904] Total x shape shape: (19968, 5, 3)
[2022-12-05 16:01:18.802989] Total y shape shape: (19968,)


# RNN

## Standard metrics - 3 sigma threshold

In [16]:
SIGMA_THRESHOLD = 3

In [17]:
sigma_threshold = get_threshold_for_sigma(model_config_rnn.pvalue_table, SIGMA_THRESHOLD)
model_config_rnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_rnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_rnn.pvalue_table, model_config_rnn.ad.threshold)} sigma")

Threshold: 0.0509078449 corresponding to 3.0083403183 sigma


In [18]:
print(f"************** Evaluating {model_config_rnn.name} patience={model_config_rnn.patience} **************")
metrics = model_config_rnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_rnn.ad.detection_delay(test_all_y, model_config_rnn.ad.predict(test_all_x), test_set_size, model_config_rnn.timesteps))

************** Evaluating AnomalyDetector_rnn_l2_u32 patience=5 **************
Using threshold 0.0509078449
accuracy 0.7890625
precision 0.9980013324450366
recall 0.5879120879120879
f1 0.7399357866139789
roc_auc 0.7933422960018706
confusion_matrix [[9764, 12], [4200, 5992]]
false_positive_rate 0.001998667554963358
detection_delay: 6.576923076923077


## Standard metrics - 5 sigma threshold

In [22]:
SIGMA_THRESHOLD = 4.5

In [23]:
sigma_threshold = get_threshold_for_sigma(model_config_rnn.pvalue_table, SIGMA_THRESHOLD)
model_config_rnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_rnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_rnn.pvalue_table, model_config_rnn.ad.threshold)} sigma")

Threshold: 0.1540617529 corresponding to 4.5564896349 sigma


In [24]:
print(f"************** Evaluating {model_config_rnn.name} patience={model_config_rnn.patience} **************")
metrics = model_config_rnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_rnn.ad.detection_delay(test_all_y, model_config_rnn.ad.predict(test_all_x), test_set_size, model_config_rnn.timesteps))

************** Evaluating AnomalyDetector_rnn_l2_u32 patience=5 **************
Using threshold 0.1540617529
accuracy 0.7433894230769231
precision 0.998426435877262
recall 0.4980376766091052
f1 0.6645718774548312
roc_auc 0.7486096730017703
confusion_matrix [[9768, 8], [5116, 5076]]
false_positive_rate 0.0015735641227380016
detection_delay: 4.759615384615385


## Sigma table

In [25]:
data = {}
for template in tqdm(list(data_manager.data.keys())):
    test_x, test_y = data_manager.get_test_set(template=template, onset=250, integration_time=5, sub_window_size=5, stride=1)
    model_config_rnn.ad.predict(test_x)
    data[template] = get_sigma_for_ts_array(model_config_rnn.pvalue_table, model_config_rnn.ad.get_reconstruction_errors(), verbose=False)

sigma_table_rnn = pd.DataFrame(data, index=DataManager.get_tbin_of_sequences(96, 5, 5, 1))    

100%|█████████████████████████████████████████| 208/208 [00:11<00:00, 18.36it/s]


In [26]:
sigma_table_rnn

,run0002_ID000044,run0011_ID000139,run0016_ID000340,run0017_ID000132,run0017_ID000261,run0019_ID000070,run0019_ID000268,run0025_ID000471,run0026_ID000127,run0029_ID000397,run0029_ID000446,run0032_ID000190,run0038_ID000228,run0043_ID000231,run0043_ID000324,run0043_ID000460,run0046_ID000029,run0046_ID000300,run0048_ID000465,run0051_ID000166,run0051_ID000434,run0055_ID000128,run0066_ID000041,run0067_ID000395,run0076_ID000465,run0077_ID000381,run0079_ID000383,run0096_ID000038,run0099_ID000496,run0101_ID000172,run0109_ID000136,run0111_ID000027,run0113_ID000112,run0114_ID000299,run0115_ID000189,run0118_ID000382,run0118_ID000393,run0121_ID000156,run0130_ID000461,run0138_ID000218,run0142_ID000123,run0146_ID000301,run0147_ID000002,run0150_ID000189,run0154_ID000068,run0155_ID000328,run0156_ID000028,run0157_ID000407,run0158_ID000220,run0158_ID000313,run0160_ID000149,run0163_ID000414,run0173_ID000131,run0176_ID000473,run0180_ID000255,run0181_ID000296,run0182_ID000054,run0183_ID000331,run0191_ID000005,run0192_ID000017,run0192_ID000068,run0193_ID000134,run0197_ID000145,run0199_ID000215,run0199_ID000278,run0210_ID000082,run0211_ID000462,run0214_ID000101,run0218_ID000245,run0221_ID000122,run0222_ID000318,run0226_ID000257,run0231_ID000152,run0231_ID000320,run0233_ID000376,run0240_ID000371,run0242_ID000066,run0252_ID000066,run0254_ID000456,run0258_ID000139,run0258_ID000165,run0266_ID000397,run0278_ID000152,run0279_ID000405,run0283_ID000065,run0284_ID000362,run0286_ID000021,run0286_ID000099,run0291_ID000001,run0294_ID000203,run0301_ID000376,run0309_ID000061,run0315_ID000129,run0317_ID000219,run0318_ID000143,run0325_ID000165,run0339_ID000132,run0339_ID000241,run0340_ID000060,run0341_ID000126,run0343_ID000134,run0344_ID000129,run0349_ID000154,run0350_ID000189,run0352_ID000076,run0354_ID000089,run0358_ID000063,run0363_ID000327,run0369_ID000392,run0370_ID000146,run0382_ID000397,run0383_ID000191,run0385_ID000009,run0395_ID000138,run0398_ID000008,run0403_ID000285,run0407_ID000101,run0410_ID000425,run0420_ID000152,run0421_ID000208,run0421_ID000212,run0423_ID000024,run0428_ID000157,run0429_ID000078,run0431_ID000202,run0436_ID000045,run0444_ID000110,run0452_ID000030,run0453_ID000359,run0457_ID000240,run0459_ID000351,run0460_ID000344,run0461_ID000109,run0467_ID000192,run0478_ID000180,run0478_ID000222,run0478_ID000299,run0481_ID000384,run0493_ID000305,run0494_ID000343,run0496_ID000386,run0498_ID000059,run0498_ID000071,run0498_ID000373,run0501_ID000313,run0501_ID000398,run0508_ID000316,run0524_ID000291,run0533_ID000171,run0535_ID000181,run0537_ID000149,run0556_ID000312,run0559_ID000332,run0562_ID000069,run0563_ID000044,run0569_ID000136,run0576_ID000397,run0583_ID000329,run0584_ID000213,run0586_ID000161,run0592_ID000035,run0607_ID000306,run0610_ID000063,run0610_ID000260,run0615_ID000207,run0615_ID000245,run0620_ID000109,run0629_ID000224,run0637_ID000089,run0641_ID000127,run0641_ID000226,run0642_ID000033,run0652_ID000307,run0653_ID000037,run0655_ID000259,run0658_ID000260,run0667_ID000126,run0684_ID000346,run0688_ID000044,run0697_ID000167,run0709_ID000364,run0711_ID000203,run0712_ID000163,run0722_ID000130,run0722_ID000187,run0731_ID000202,run0762_ID000202,run0763_ID000224,run0765_ID000033,run0767_ID000231,run0780_ID000007,run0788_ID000236,run0792_ID000290,run0793_ID000465,run0798_ID000122,run0798_ID000403,run0799_ID000140,run0804_ID000031,run0808_ID000191,run0811_ID000253,run0812_ID000337,run0839_ID000040,run0842_ID000212,run0853_ID000363,run0879_ID000213,run0880_ID000380,run0881_ID000086,run0885_ID000163
0-25,0.000,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0.000000,0,0,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.00000,0.0

# CNN

## Standard metrics - 3 sigma

In [ ]:
SIGMA_THRESHOLD = 3

In [ ]:
sigma_threshold = get_threshold_for_sigma(model_config_cnn.pvalue_table, SIGMA_THRESHOLD)
model_config_cnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_cnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_cnn.pvalue_table, model_config_cnn.ad.threshold)} sigma")

In [ ]:
print(f"************** Evaluating {model_config_cnn.name} patience={model_config_cnn.patience} **************")
metrics = model_config_cnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_cnn.ad.detection_delay(test_all_y, model_config_cnn.ad.predict(test_all_x), test_set_size, model_config_cnn.timesteps))

## Standard metrics - 5 sigma

In [ ]:
SIGMA_THRESHOLD = 5

In [ ]:
sigma_threshold = get_threshold_for_sigma(model_config_cnn.pvalue_table, SIGMA_THRESHOLD)
model_config_cnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_cnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_cnn.pvalue_table, model_config_cnn.ad.threshold)} sigma")

In [ ]:
print(f"************** Evaluating {model_config_cnn.name} patience={model_config_cnn.patience} **************")
metrics = model_config_cnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_cnn.ad.detection_delay(test_all_y, model_config_cnn.ad.predict(test_all_x), test_set_size, model_config_cnn.timesteps))

## Sigma table

In [ ]:
data = {}
for template in tqdm(list(data_manager.data.keys())):
    test_x, test_y = data_manager.get_test_set(template=template, onset=250, integration_time=5, sub_window_size=5, stride=1)
    model_config_cnn.ad.predict(test_x)
    data[template] = get_sigma_for_ts_array(model_config_cnn.pvalue_table, model_config_cnn.ad.get_reconstruction_errors(), verbose=False)

sigma_table_cnn = pd.DataFrame(data, index=DataManager.get_tbin_of_sequences(96, 5, 5, 1))    

In [ ]:
sigma_table_cnn

# Li&Ma

In [ ]:
li_ma_data = {}
templates = list(data_manager.data.keys())
templates = []
for template in tqdm(templates):
    lima_df = LiMa.detect("binned", DataManager.get_fits_from_template(fits_files, template), temporal_bins=sigma_table_cnn.index.values, tobs=500, sigma_gt=0)
    li_ma_data[template] = lima_df["sigma"].values
lima_sigma_table = pd.DataFrame(li_ma_data, index=sigma_table_cnn.index.values)
if len(templates) > 0:
    lima_sigma_table.to_csv(Path(output_dir).joinpath("li_ma_detection_table.csv"))    

In [ ]:
lima_sigma_table = pd.read_csv(Path(output_dir).joinpath("li_ma_detection_table.csv"), index_col=0)

In [ ]:
lima_sigma_table

## Results

In [ ]:
templates_detections = dotdict({})
for templ_det in lima_sigma_table:
    templates_detections[templ_det] = dotdict({}) 
    templates_detections[templ_det]["li_ma"] = dotdict({})
    templates_detections[templ_det]["cnn"] = dotdict({})
    templates_detections[templ_det]["rnn"] = dotdict({})

In [ ]:
for templ_det in lima_sigma_table:
    detections_3s = lima_sigma_table[templ_det] >= 3
    detections_5s = lima_sigma_table[templ_det] >= 5
    templates_detections[templ_det]["li_ma"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["li_ma"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["li_ma"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["li_ma"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]

for templ_det in sigma_table_rnn:
    detections_3s = sigma_table_rnn[templ_det] >= 3
    detections_5s = sigma_table_rnn[templ_det] >= 5
    templates_detections[templ_det]["rnn"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["rnn"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["rnn"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["rnn"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]
    
for templ_det in sigma_table_cnn:
    detections_3s = sigma_table_cnn[templ_det] >= 3
    detections_5s = sigma_table_cnn[templ_det] >= 5
    templates_detections[templ_det]["cnn"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["cnn"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["cnn"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["cnn"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]    

In [ ]:
def get_dd(model, templates_detections, onset_index=45):
    first_detection_indexes = []
    for template_result in templates_detections.values():
        if len(template_result[model]["3s_detections_indexes"]) > 0:
            first_detection_indexes.append(template_result[model]["3s_detections_indexes"][0])
    return round(np.array(first_detection_indexes).mean()-onset_index,2)

### 3 sigma detections

In [ ]:
print(f'Detected by CNN: {np.array([template_result["cnn"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("cnn",templates_detections)} s')
print(f'Detected by RNN: {np.array([template_result["rnn"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("rnn",templates_detections)} s')
print(f'Detected by Li&Ma: {np.array([template_result["li_ma"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("li_ma",templates_detections)} s')

### 5 sigma detections

In [ ]:
print(f'Detected by CNN: {np.array([template_result["cnn"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("cnn",templates_detections)} s')
print(f'Detected by RNN: {np.array([template_result["rnn"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("rnn",templates_detections)} s')
print(f'Detected by Li&Ma: {np.array([template_result["li_ma"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("li_ma",templates_detections)} s')

### Sigma plots RNN vs Li&Ma

In [ ]:
matplotlib.use("TKAgg", force=True)
%matplotlib inline
from rtapipe.lib.plotting.PlotConfig import PlotConfig

def sigma_plot(template, sigma_table_model, lima_sigma_table):
    pc = PlotConfig()    
    fig, ax = plt.subplots(1,1, figsize=pc.fig_size)
    ax.plot(sigma_table_model.index.values, sigma_table_model[template], marker='o', linestyle='--', label="Anomaly Detection")
    ax.scatter(sigma_table_model.index.values, lima_sigma_table[template], marker='x', label="Li&Ma")
    ax.tick_params(axis='y', labelsize=15, width=2, grid_alpha=0.5)
    ax.tick_params(axis='x', which='major', labelsize=10)
    fig.suptitle("Detection plot")
    ax.set_title(f"Template {template}")
    ax.set_ylabel("Sigma")
    ax.set_xlabel("Timeseries")
    ax.set_ylim(0, 10)
    plt.axvline(x = 45, color = 'grey', linestyle="-.")
    plt.axhline(y = 3, color = 'orange', linestyle="-.")
    plt.axhline(y = 5, color = 'red', linestyle="-.")
    mask = [True if i%2==0 else False for i in range(len(sigma_table_model.index.values))]
    ax.set_xticks(range(0, len(sigma_table_model.index.values), 2), sigma_table_model.index.values[mask], rotation=45)
    ax.legend()
    fig.savefig(Path(output_dir).joinpath(f"detection_plot_{template}.png"), dpi=pc.dpi)
    plt.show()    
    
for template in tqdm(templates):
    sigma_plot(template, sigma_table_rnn, lima_sigma_table)